# Cybersecurity advanced
## Lab 04: Suricata


Links to Syllabus

- [Theoretical](https://hogenttin.github.io/cybersecurity-advanced/lesson-4/theory/)
- [Practical](https://hogenttin.github.io/cybersecurity-advanced/lesson-14lab/)

### Suricata

First we disable nftables from previous lab

To install suricata:

```bash
sudo dnf install epel-release -y
sudo dnf update -y
sudo dnf install suricata -y
```

Then create a rules file called local.rules in the /etc/suricata/rules directory
The file contains following ruleset:

```bash
      alert icmp any any -> any any (msg:"ICMP Ping detected"; sid:1000001; rev:1;)
      alert tcp any any -> any 3306 (msg:"MySQL Connection Detected"; sid:1000002; rev:1;)
```

After change the ruleset directory and rules in the `suricata.yml`

```bash
default-rule-path: /etc/suricata/rules
      
rule-files:
- local.rules
```


Suricata still listens on ETH0, you can check this using `sudo systemctl status suricata`, it will show:

```bash
● suricata.service - Suricata Intrusion Detection Service
     Loaded: loaded (/usr/lib/systemd/system/suricata.service; enabled; preset: disabled)
     Active: active (running) since Fri 2024-12-27 14:49:10 UTC; 8min ago
       Docs: man:suricata(1)
    Process: 944 ExecStartPre=/bin/rm -f /var/run/suricata.pid (code=exited, status=0/SUCCESS)
   Main PID: 946 (Suricata-Main)
      Tasks: 7 (limit: 24468)
     Memory: 42.4M
        CPU: 3.992s
     CGroup: /system.slice/suricata.service
             └─946 /sbin/suricata -c /etc/suricata/suricata.yaml --pidfile /var/run/suricata.pid -i eth0 --user suricata
```

We need to change this to eth1
change the eth0 to eth1 in the `/etc/sysconfig/suricata`

After we restart suricata `sudo systemctl restart suricata`
if we check the status again the nic is changed to eth1:

```bash
● suricata.service - Suricata Intrusion Detection Service
     Loaded: loaded (/usr/lib/systemd/system/suricata.service; enabled; preset: disabled)
     Active: active (running) since Fri 2024-12-27 14:52:10 UTC; 8min ago
       Docs: man:suricata(1)
    Process: 944 ExecStartPre=/bin/rm -f /var/run/suricata.pid (code=exited, status=0/SUCCESS)
   Main PID: 946 (Suricata-Main)
      Tasks: 7 (limit: 24468)
     Memory: 42.4M
        CPU: 3.992s
     CGroup: /system.slice/suricata.service
             └─946 /sbin/suricata -c /etc/suricata/suricata.yaml --pidfile /var/run/suricata.pid -i eth1 --user suricata
```

After this change we start a ping on the ***red*** machine `ping 172.30.0.4` and wait for a couple of minutes

We check the fast.log file on the companyrouter, `sudo cat /var/log/suricata/fast.log`

```bash
[root@companyrouter suricata]# cat /var/log/suricata/fast.log
12/27/2024-14:52:11.365516  [**] [1:1000001:1] ICMP Ping detected [**] [Classification: (null)] [Priority: 3] {ICMP} 192.168.62.10:8 -> 172.30.0.4:0
12/27/2024-14:52:11.365889  [**] [1:1000001:1] ICMP Ping detected [**] [Classification: (null)] [Priority: 3] {ICMP} 172.30.0.4:0 -> 192.168.62.10:0
12/27/2024-14:53:00.264770  [**] [1:1000001:1] ICMP Ping detected [**] [Classification: (null)] [Priority: 3] {IPv6-ICMP} fe80:0000:0000:0000:0a00:27ff:fe5b:dd1e:133 -> ff02:0000:0000:0000:0000:0000:0000:0002:0
12/27/2024-14:59:09.984563  [**] [1:1000001:1] ICMP Ping detected [**] [Classification: (null)] [Priority: 3] {IPv6-ICMP} fe80:0000:0000:0000:89f2:eab5:43d1:5937:143 -> ff02:0000:0000:0000:0000:0000:0000:0016:0
12/27/2024-15:01:05.449610  [**] [1:1000001:1] ICMP Ping detected [**] [Classification: (null)] [Priority: 3] {IPv6-ICMP} fe80:0000:0000:0000:0a00:27ff:fe40:efb3:133 -> ff02:0000:0000:0000:0000:0000:0000:0002:0
12/27/2024-15:01:44.553344  [**] [1:1000001:1] ICMP Ping detected [**] [Classification: (null)] [Priority: 3] {IPv6-ICMP} fe80:0000:0000:0000:0a00:27ff:fe5b:dd1e:133 -> ff02:0000:0000:0000:0000:0000:0000:0002:0
12/27/2024-15:08:24.153548  [**] [1:1000001:1] ICMP Ping detected [**] [Classification: (null)] [Priority: 3] {IPv6-ICMP} fe80:0000:0000:0000:bd42:065b:92b1:fc31:133 -> ff02:0000:0000:0000:0000:0000:0000:0002:0
```

To make it an IPS:

change the config in the /etc/suricata/suricata.yml

```bash
# Linux high speed capture support
af-packet:
  - interface: eth0
    # Number of receive threads. "auto" uses the number of cores
    threads: auto
    # Default clusterid. AF_PACKET will load balance packets based on flow.
    cluster-id: 99
    copy-mode: ips
    copy-iface: eth0
```